In [2]:
!pip -q install ipynbname
!pip -q install pandas numpy ipynbname scikit-learn
!pip install optuna
!pip install catboost


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 8.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 751.2/751.2 kB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 2.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 11.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.5/599.5 kB 44.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --up

In [3]:
def drop_cols_with_na(train,val):
    cat_cols_with_na = [
        #'세부 시술 유형', ##2
        '이전 총 임신 횟수', ##11703
        '이전 총 임신 성공 횟수', ##5081
        
        '총 생성 배아 수', ## DI
        '저장된 배아 수', ## DI
        '채취된 신선 난자 수', ## DI
        '수정 시도된 난자 수' ## DI
    ]

    # numeric_cols_with_na = [
    #     '이식된 배아 수', ## DI
    #     '미세주입(ICSI) 배아 이식 수', ## 8890
    #     '배아 이식 후 경과일', ## 17440
    # ]
    cols = cat_cols_with_na #+ numeric_cols_with_na
    train = train.drop(columns=cols,axis=1)
    val = val.drop(columns=cols,axis=1)
    return train, val
def numeric_nan(train,val):
    numeric_cols_with_na = [
        '이식된 배아 수', ## DI
        '미세주입(ICSI) 배아 이식 수', ## 8890
        '배아 이식 후 경과일', ## 17440 #얘는 따로 8.0 또는 -1로 둬보기?
    ]
    # for col in numeric_cols_with_na:
    #     train[col] = train[col].fillna(0)
    #     val[col] = val[col].fillna(0)
    train['이식된 배아 수'] = train['이식된 배아 수'].fillna(2)
    val['이식된 배아 수'] = val['이식된 배아 수'].fillna(2)
    
    # train['미세주입(ICSI) 배아 이식 수'] = train['미세주입(ICSI) 배아 이식 수'].fillna(0)
    # val['미세주입(ICSI) 배아 이식 수'] = val['미세주입(ICSI) 배아 이식 수'].fillna(0)
    # train['배아 이식 후 경과일'] = train['배아 이식 후 경과일'].fillna(-1)
    # val['배아 이식 후 경과일'] = val['배아 이식 후 경과일'].fillna(-1)
    train.loc[(train['시술 유형'] == 'IVF') & (train['미세주입(ICSI) 배아 이식 수'].isna())].fillna(0)
    val.loc[(val['시술 유형'] == 'IVF') & (val['미세주입(ICSI) 배아 이식 수'].isna())].fillna(0)
    train.loc[(train['시술 유형'] == 'DI') & (train['미세주입(ICSI) 배아 이식 수'].isna())].fillna(4)
    val.loc[(val['시술 유형'] == 'DI') & (val['미세주입(ICSI) 배아 이식 수'].isna())].fillna(4)

    train.loc[(train['시술 유형'] == 'IVF') & (train['배아 이식 후 경과일'].isna())].fillna(8)
    val.loc[(val['시술 유형'] == 'IVF') & (val['배아 이식 후 경과일'].isna())].fillna(8)
    train.loc[(train['시술 유형'] == 'DI') & (train['배아 이식 후 경과일'].isna())].fillna(5)
    val.loc[(val['시술 유형'] == 'DI') & (val['배아 이식 후 경과일'].isna())].fillna(5)
    return train, val
def numeric_nan(train, val):
    # '이식된 배아 수'의 최빈값을 학습 데이터에서 계산 후 결측치 대체
    mode_embryo = train['이식된 배아 수'].mode()[0]
    train['이식된 배아 수'] = train['이식된 배아 수'].fillna(mode_embryo)
    val['이식된 배아 수'] = val['이식된 배아 수'].fillna(mode_embryo)
    
    # 미세주입(ICSI) 배아 이식 수: 시술 유형에 따라 결측치 대체
    mode_embryo_미세 = train['미세주입(ICSI) 배아 이식 수'].mode()[0]
    train.loc[(train['시술 유형'] == 'IVF') & (train['미세주입(ICSI) 배아 이식 수'].isna()), '미세주입(ICSI) 배아 이식 수'] = mode_embryo_미세
    val.loc[(val['시술 유형'] == 'IVF') & (val['미세주입(ICSI) 배아 이식 수'].isna()), '미세주입(ICSI) 배아 이식 수'] = mode_embryo_미세
    train.loc[(train['시술 유형'] == 'DI') & (train['미세주입(ICSI) 배아 이식 수'].isna()), '미세주입(ICSI) 배아 이식 수'] = 4
    val.loc[(val['시술 유형'] == 'DI') & (val['미세주입(ICSI) 배아 이식 수'].isna()), '미세주입(ICSI) 배아 이식 수'] = 4

    # 배아 이식 후 경과일: 시술 유형에 따라 결측치 대체
    mode_embryo_배아 = train['배아 이식 후 경과일'].mode()[0]
    train.loc[(train['시술 유형'] == 'IVF') & (train['배아 이식 후 경과일'].isna()), '배아 이식 후 경과일'] = 8
    val.loc[(val['시술 유형'] == 'IVF') & (val['배아 이식 후 경과일'].isna()), '배아 이식 후 경과일'] = 8
    train.loc[(train['시술 유형'] == 'DI') & (train['배아 이식 후 경과일'].isna()), '배아 이식 후 경과일'] = mode_embryo_배아
    val.loc[(val['시술 유형'] == 'DI') & (val['배아 이식 후 경과일'].isna()), '배아 이식 후 경과일'] = mode_embryo_배아
    
    return train, val


def 횟수_to_int(df_train, df_val):
    for col in [col for col in df_train.columns if '횟수' in col]:
        df_train[col] = df_train[col].replace({'6회 이상': '6회'})
        df_val[col] = df_val[col].replace({'6회 이상': '6회'})

        df_train[col] = df_train[col].str[0].astype(int)
        df_val[col] = df_val[col].str[0].astype(int)

    return df_train, df_val


def 시술유형(train, test):
    train['세부 시술 유형'] = train['세부 시술 유형'].fillna("Unknown")
    test['세부 시술 유형'] = test['세부 시술 유형'].fillna("Unknown")

    def categorize_procedure(proc):
        tokens = [token.strip() for token in proc.split(",") if token.strip() and not token.strip().isdigit()]
        # 우선순위에 따른 범주화
        if tokens.count("Unknown") >= 1:
            return "Unknown"
        if tokens.count("AH") >= 1:
            return "AH"
        if tokens.count("BLASTOCYST") >= 1:
            return "BLASTOCYST"
        if tokens.count("ICSI") >= 2 or tokens.count("IVF") >= 2:
            return "2ICSI_2IVF"
        if tokens.count("IVF") >= 1 and tokens.count("ICSI") >= 1:
            return "IVF_ICSI"
        if tokens == "ICSI":
            return "ICSI"
        if tokens == "IVF":
            return "IVF"
        return ",".join(tokens) if tokens else None

    for df in [train, test]:
        df['세부 시술 유형'] = df['세부 시술 유형'].str.replace(" / ", ",")
        df['세부 시술 유형'] = df['세부 시술 유형'].str.replace(":", ",")
        df['세부 시술 유형'] = df['세부 시술 유형'].str.replace(" ", "")

    counts = train['세부 시술 유형'].value_counts()
    allowed_categories = counts[counts >= 100].index.tolist()

    # allowed_categories에 속하지 않는 값은 "Unknown"으로 대체
    train.loc[~train['세부 시술 유형'].isin(allowed_categories), '세부 시술 유형'] = "Unknown"
    test.loc[~test['세부 시술 유형'].isin(allowed_categories), '세부 시술 유형'] = "Unknown"

    train['세부 시술 유형'] = train['세부 시술 유형'].apply(categorize_procedure)
    test['세부 시술 유형'] = test['세부 시술 유형'].apply(categorize_procedure)

    train['시술유형_통합'] = train['시술 유형'].astype(str) + '_' + train['세부 시술 유형'].astype(str)
    test['시술유형_통합'] = test['시술 유형'].astype(str) + '_' + test['세부 시술 유형'].astype(str)

    drop_cols = ['시술 유형', '세부 시술 유형']
    train = train.drop(drop_cols, axis=1)
    test = test.drop(drop_cols, axis=1)

    return train, test

def 단일배아이식여부(df_train, df_val):
    df_train['단일 배아 이식 여부'] = df_train['단일 배아 이식 여부'].fillna(0)
    df_val['단일 배아 이식 여부'] = df_val['단일 배아 이식 여부'].fillna(0)
    return df_train, df_val



def categorical_encoding(train,val, categorical_columns):

    # OrdinalEncoder를 사용하여 카테고리형 변수 인코딩
    X_train_encoded = train.copy()

    ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
    X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(train[categorical_columns])

    X_test_encoded = val.copy()
    X_test_encoded[categorical_columns] = ordinal_encoder.transform(val[categorical_columns])

    return X_train_encoded, X_test_encoded

def numeric_encoding(train,val, numeric_columns):
    # 수치형 변수 중 결측치가 있는 경우 0으로 채움
    train[numeric_columns] = train[numeric_columns].fillna(0)
    val[numeric_columns] = val[numeric_columns].fillna(0)
    return train, val

def bool_encoding(train,val,bool_features):
    # 불리언형 변수는 0과 1로 변환
    train[bool_features] = train[bool_features].fillna(False).astype(int)
    val[bool_features] = val[bool_features].fillna(False).astype(int)
    return train, val

def check_cols(train,val):

    cols_without_na = train.columns
    categorical_columns = ['환자 시술 당시 나이', '시술 유형', '세부 시술 유형', '이전 IVF 시술 횟수',
       '이전 DI 시술 횟수', '이전 총 임신 횟수', '이전 총 임신 성공 횟수', '총 생성 배아 수', '저장된 배아 수',
       '해동된 배아 수', '채취된 신선 난자 수', '수정 시도된 난자 수', '난자 출처', '정자 출처', '난자 기증자 나이',
       '정자 기증자 나이']

    numeric_columns = ['이식된 배아 수','미세주입(ICSI) 배아 이식 수','배아 이식 후 경과일']

    bool_features = ['배란 자극 시술 여부', '단일 배아 이식 여부', '불임 원인 - 난관 질환',
    '불임 원인 - 배란 장애', '불임 원인 - 남성 요인', '불임 원인 - 자궁내막증', '불임 원인 - 불명확', '해동 난자 사용 여부',
    '신선 난자 사용 여부', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부', '착상 전 PGD 시행 여부', '착상 전 PGS 시행 여부']
    
    categorical_columns_cleaned = [col for col in categorical_columns if col in cols_without_na]
    numeric_columns_cleaned = [col for col in numeric_columns if col in cols_without_na]
    bool_features_cleaned = [col for col in bool_features if col in cols_without_na]
    
    # print("Categorical columns:", categorical_columns_cleaned, "\n count", len(categorical_columns_cleaned))
    # print("Numeric columns:", numeric_columns_cleaned, "\n count", len(numeric_columns_cleaned))
    # print("Boolean columns:", bool_features_cleaned, "\n count", len(bool_features_cleaned))

    return categorical_columns_cleaned, numeric_columns_cleaned, bool_features_cleaned

def label_encoding(train, test, cols):
    encoder = LabelEncoder()
    for col in cols:
        train[col] = encoder.fit_transform(train[col])
        test[col] = encoder.transform(test[col])
    return train, test


def type_to_category(train, test, cols):
    train[cols] = train[cols].astype('category')
    test[cols] = test[cols].astype('category')
    return train, test


def num_feature_scailing(train, test, seed=777):
    cat_cols = [col for col in train.columns if pd.api.types.is_categorical_dtype(train[col])]
    numeric_cols = [col for col in train.columns if col not in cat_cols and col != '임신 성공 확률']
    # bin_cols 들도 동일하게 스케일링

    arr_train = train[numeric_cols].to_numpy()  # DataFrame -> NumPy
    arr_train = arr_train.astype(np.float32)
    arr_test = test[numeric_cols].to_numpy()
    arr_test = arr_test.astype(np.float32)

    np.random.seed(seed)
    random.seed(seed)
    noise = (
        np.random.default_rng(0)
        .normal(0.0, 1e-5, arr_train.shape)
        .astype(arr_train.dtype)
    )
    preprocessing = QuantileTransformer(
        n_quantiles=max(min(len(train[numeric_cols]) // 30, 1000), 10),
        output_distribution='normal',
        subsample=10**9,
    ).fit(arr_train + noise)

    train[numeric_cols] = preprocessing.transform(arr_train)
    test[numeric_cols] = preprocessing.transform(arr_test)
    return train, test

def drop_single_value_columns(df_train, df_test):
    cols_to_drop = [col for col in df_train.columns if df_train[col].nunique() == 1]
    return df_train.drop(columns=cols_to_drop), df_test.drop(columns=cols_to_drop)

def 이전_총_임신_성공_횟수(train, test):
    train['이전 총 임신 횟수'] = train['이전 총 임신 횟수'].fillna(train['이전 총 임신 횟수'].mode()[0])
    test['이전 총 임신 횟수'] = test['이전 총 임신 횟수'].fillna(train['이전 총 임신 횟수'].mode()[0])

    train['이전 총 임신 성공 횟수'] = train['이전 총 임신 성공 횟수'].fillna(train['이전 총 임신 성공 횟수'].mode()[0])
    test['이전 총 임신 성공 횟수'] = test['이전 총 임신 성공 횟수'].fillna(train['이전 총 임신 성공 횟수'].mode()[0])

def 독립범주로보기(train, test):
    cols = ['총 생성 배아 수', '저장된 배아 수', '채취된 신선 난자 수', '수정 시도된 난자 수']
    for col in cols:
        train[col] = train[col].fillna('NAN')
        test[col] = test[col].fillna('NAN')
        
        
def all_preprocess(train,val):
    이전_총_임신_성공_횟수(train, val)
    독립범주로보기(train, val)
    #train, val = drop_cols_with_na(train, val)
    train, val = numeric_nan(train, val)
    # 기본 전처리 단계
    train, val = 횟수_to_int(train, val)

    train, val = 시술유형(train, val)
    # train, val = 임신_IVF(train, val)

    train, val = 단일배아이식여부(train, val)

    cols_to_encoding = [
        "환자 시술 당시 나이",
        # "클리닉 내 총 시술 횟수",
        # "IVF 시술 횟수",
        # "DI 시술 횟수",
        # "총 임신 횟수",
        # "IVF 임신 횟수",
        # "DI 임신 횟수",
        # "총 출산 횟수",
        # "IVF 출산 횟수",
        # "DI 출산 횟수",
        "난자 출처",
        "정자 출처",
        "난자 기증자 나이",
        "정자 기증자 나이",
        '시술유형_통합',

        '해동된 배아 수', # 원래 int였는데 범주형으로 바뀜
        '총 생성 배아 수',
        '저장된 배아 수', 
        '채취된 신선 난자 수', 
        '수정 시도된 난자 수'

    ]
    # categorical_columns, numeric_columns, bool_features=check_cols(train,val)
    
    # train, val = categorical_encoding(train,val, categorical_columns)
    # train, val= numeric_encoding(train,val, numeric_columns)
    # train, val = bool_encoding(train,val,bool_features)

    train, val = label_encoding(train, val, cols=cols_to_encoding)
    train, val = type_to_category(train, val, cols=cols_to_encoding)

    # train, val = impute_nan(train, val)
    train, val = num_feature_scailing(train, val)

    train, val = drop_single_value_columns(train, val)
    
    return train, val
        

In [5]:
train_path = "/home/elicer/LG_Aimers_6th/offline_data/train_aimers_6th_offline.csv"
test_path = "/home/elicer/LG_Aimers_6th/offline_data/test_aimers_6th_offline.csv"


In [6]:
import optuna
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor
from 평가_Metric import competition_metric, f1_score, weighted_brier_score
from sklearn.preprocessing import LabelEncoder, StandardScaler, QuantileTransformer, MultiLabelBinarizer, OrdinalEncoder

seed = 333
train = pd.read_csv(train_path, encoding='utf-8-sig').drop(columns=["ID"])
test = pd.read_csv(test_path, encoding='utf-8-sig').drop(columns=["ID"])

# 범주형 변수로 처리할 열 목록
cols_to_encoding = [
    "환자 시술 당시 나이",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "시술유형_통합",
    "해동된 배아 수",  # 원래 int였으나 범주형으로 처리
    "총 생성 배아 수",
    "저장된 배아 수",
    "채취된 신선 난자 수",
    "수정 시도된 난자 수"
]

def objective(trial):
    # Optuna가 탐색할 하이퍼파라미터 공간 설정
    params = {
        "iterations": trial.suggest_int("iterations", 500, 2000),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.001, 0.1),
        "depth": trial.suggest_int("depth", 4, 12),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 0.001, 10.0, log=True),
        "od_type": 'Iter',
        "od_wait": trial.suggest_int("od_wait", 50, 200),
        "random_seed": seed,
        "verbose": 0
    }
    
    # 5-Fold 교차 검증으로 모델 성능 평가
    kf = KFold(n_splits=5, shuffle=True, random_state=seed)
    cv_scores = []
    
    for train_idx, valid_idx in kf.split(train):
        fold_train = train.iloc[train_idx].copy().reset_index(drop=True)
        fold_valid = train.iloc[valid_idx].copy().reset_index(drop=True)
        fold_train2 = fold_train.copy()  # 전처리 전 복사본
        
        # 사용자 정의 전처리 함수 적용
        fold_train, fold_valid = all_preprocess(fold_train, fold_valid)
        
        X_train = fold_train.drop(columns=["임신 성공 확률"])
        y_train = fold_train["임신 성공 확률"]
        X_val   = fold_valid.drop(columns=["임신 성공 확률"])
        y_val   = fold_valid["임신 성공 확률"]
        
        # 클래스 불균형 해결: 0.5 기준 이진화 후, 양성 샘플에 대한 가중치 계산
        y_train_binary = (y_train > 0.5).astype(int)
        pos_count = np.sum(y_train_binary)
        neg_count = len(y_train_binary) - pos_count
        scale_pos_weight = neg_count / pos_count if pos_count > 0 else 1.0
        sample_weights = np.where(y_train_binary == 1, scale_pos_weight, 1.0)
        
        model = CatBoostRegressor(**params)
        model.fit(
            X_train, y_train,
            sample_weight=sample_weights,
            eval_set=(X_val, y_val),
            cat_features=cols_to_encoding
        )
        
        y_val_pred = np.clip(model.predict(X_val), 0, 1)
        score = competition_metric(y_val, y_val_pred)
        cv_scores.append(score)
        print('------------------------------------------------------')
        print(cv_scores)
        print('------------------------------------------------------')
        
    return np.mean(cv_scores)

# Optuna 스터디 생성 (최소화 문제)
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

# 최적의 하이퍼파라미터 출력
print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-04-05 21:21:08,318] A new study created in memory with name: no-name-14c17508-46f5-41bb-8f55-dde4b95b179b
/tmp/ipykernel_1122/2720983545.py:33: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 0.001, 0.1),
/tmp/ipykernel_1122/4181465775.py:200: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  cat_cols = [col for col in train.columns if pd.api.types.is_categorical_dtype(train[col])]


------------------------------------------------------
[0.6543219899804318]
------------------------------------------------------


/tmp/ipykernel_1122/4181465775.py:200: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  cat_cols = [col for col in train.columns if pd.api.types.is_categorical_dtype(train[col])]


------------------------------------------------------
[0.6543219899804318, 0.6528862945978632]
------------------------------------------------------


/tmp/ipykernel_1122/4181465775.py:200: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  cat_cols = [col for col in train.columns if pd.api.types.is_categorical_dtype(train[col])]


...

------------------------------------------------------
[0.6512757980599935, 0.6509269702624135, 0.6512312738694532]
------------------------------------------------------


/tmp/ipykernel_1122/4181465775.py:200: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  cat_cols = [col for col in train.columns if pd.api.types.is_categorical_dtype(train[col])]


------------------------------------------------------
[0.6512757980599935, 0.6509269702624135, 0.6512312738694532, 0.6507741230374011]
------------------------------------------------------


/tmp/ipykernel_1122/4181465775.py:200: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  cat_cols = [col for col in train.columns if pd.api.types.is_categorical_dtype(train[col])]
[I 2025-04-05 23:00:34,085] Trial 19 finished with value: 0.651630882582039 and parameters: {'iterations': 1795, 'learning_rate': 0.0014020359718980744, 'depth': 7, 'l2_leaf_reg': 0.026976294270848234, 'od_wait': 119}. Best is trial 16 with value: 0.6562821577910213.


------------------------------------------------------
[0.6512757980599935, 0.6509269702624135, 0.6512312738694532, 0.6507741230374011, 0.6539462476809338]
------------------------------------------------------
Best trial:
  Value:  0.6562821577910213
  Params:
    iterations: 1799
    learning_rate: 0.011885681238988397
    depth: 7
    l2_leaf_reg: 0.04445003403694515
    od_wait: 97
